### Create the model with with BERT ###

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

from datetime import datetime

!pip install bert-tensorflow

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import pickle


     |████████████████████████████████| 71kB 4.9MB/s 



In [0]:
#tokenizer = create_tokenizer_from_hub_module(bert_model)
tokenizer = 0

def create_tokenizer_from_hub_module(bert_model_hub):
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(bert_model_hub)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def make_features(dataset, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module( bert_model_hub, trainable=True)
  
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)

  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, SAVE_CHECKPOINTS_STEPS, label_list, LEARNING_RATE, num_train_steps, num_warmup_steps, BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
      bert_model_hub = bert_model_hub,
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config


def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN, 
			MAX_SEQ_LENGTH = 128,
			BATCH_SIZE = 32,
			LEARNING_RATE = 2e-5,
			NUM_TRAIN_EPOCHS = 3.0,
			WARMUP_PROPORTION = 0.1,
			SAVE_SUMMARY_STEPS = 100,
			SAVE_CHECKPOINTS_STEPS = 10000,
			bert_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"):

    label_list = train[LABEL_COLUMN].unique().tolist()
    
    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_model_hub, 
                                  OUTPUT_DIR, 
                                  SAVE_SUMMARY_STEPS, 
                                  SAVE_CHECKPOINTS_STEPS, 
                                  label_list, 
                                  LEARNING_RATE, 
                                  num_train_steps, 
                                  num_warmup_steps, 
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator



### Loading data (REAL FINANCIAL DATA AND FAKE NEWS) ###




In [34]:
import os
import json
import pandas as pd

#cargamos um dataset rotulado de noticias relacionadas a DJI
!wget https://raw.githubusercontent.com/stonescenter/public_data/master/text/dataset_dji_labeled.json
!wget https://raw.githubusercontent.com/stonescenter/public_data/master/text/rumours_dataset.json

# treinamos com noticas reales
DATA_TRAIN = 'dataset_dji_labeled.json'

# Loads the JSON files with news sites
with open(DATA_TRAIN) as data_file:
    news = json.load(data_file)

df = pd.DataFrame.from_dict(news, orient='columns')
df.Text =  df.Title + ' ' + df.Text 
df = df[['Text', 'label']]

df.rename(columns={'Text':'text'},inplace=True)
df.head()

,text,label
0,"U.S. and China Extend Talks, but Final Deal Re...",1
1,The Dollar Is Still King. How (in the World) D...,1
2,Fed Explains Pause as Officials Debate Future ...,-1
3,Democrats Want to Tax the Wealthy. Many Voters...,-1
4,Housing Is Already in a Slump. So It (Probably...,1


### Training model with BERT ###


In [0]:
split = 0.8
i_split = round(len(df) * split)

train = df.iloc[0:i_split,0:]
test = df.iloc[i_split:,0:]

OUTPUT_DIR = 'results/'

tf.gfile.MakeDirs(OUTPUT_DIR)
myparam = {
	"DATA_COLUMN": "text",
	"LABEL_COLUMN": "label",
	"LEARNING_RATE": 2e-5,
	"NUM_TRAIN_EPOCHS":10,
	"MAX_SEQ_LENGTH": 128
  }
label_list = df['label'].unique().tolist()

result, estimator = run_on_dfs(train, test, **myparam)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 450


INFO:tensorflow:Writing example 0 of 450


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into results/model.ckpt.


INFO:tensorflow:Saving checkpoints for 0 into results/model.ckpt.


INFO:tensorflow:loss = 0.76248544, step = 1


INFO:tensorflow:loss = 0.76248544, step = 1


In [0]:
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

# imprimimos os resultados do treino    
pretty_print(result)

### Predict ###
 Utilizaremos os dado desconhecidos de um dataset que contem fakenews

In [0]:
bert_model = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(bert_model)

def getPrediction(in_sentences):

  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, myparam['MAX_SEQ_LENGTH'], tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=myparam['MAX_SEQ_LENGTH'], is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)

  return [(sentence, prediction['probabilities'], prediction['labels']) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
#cargamos o dataset gerado de noticas do PHEME fakenews
DATA_TEST = 'rumours_dataset.json' 

with open(DATA_TEST) as data_file:
  news = json.load(data_file)

df_test = pd.DataFrame.from_dict(news, orient='columns')


df['is_fake'].replace(
    to_replace=['false'],
    value=-1,
    inplace=True
)

df_test = df_test[df_test.is_fake == 'false']

'''

#renombramos as columnas
df_test.rename(columns={'is_fake':'label'},inplace=True)

#print(df_test.columns)
df_test.head()

In [0]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!",
  "impeachment inquiry against Donald Trump"
]

In [0]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 5


INFO:tensorflow:Writing example 0 of 5


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from results/model.ckpt-426


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
predictions

[('That movie was absolutely awful',
  array([-6.5185151e+00, -1.4769847e-03], dtype=float32),
  1),
 ('The acting was a bit lacking',
  array([-0.02674716, -3.634669  ], dtype=float32),
  0),
 ('The film was creative and surprising',
  array([-0.64899874, -0.7393351 ], dtype=float32),
  0),
 ('Absolutely fantastic!',
  array([-0.92834294, -0.5028708 ], dtype=float32),
  1),
 ('impeachment inquiry against Donald Trump',
  array([-7.9144540e+00, -3.6554807e-04], dtype=float32),
  1)]